In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt
from typing import Dict, Any, Callable, Union, List

def pmf(n:int, p:float, k:Union[int, List[int], np.ndarray]) -> Union[float, np.ndarray]:
  "Probability mass function"
  q = 1-p
  k = np.asarray(k)
  # validate k is in valid range
  valid_k = (k >= 0) & (k <= n)
  result = np.zeros_like(k, dtype=float)
  k_valid = k[valid_k]
  
  if len(k_valid) > 0:
    # calculate binomial coefficient
    binomial_coeff = np.array([math.comb(n, int(ki)) for ki in k_valid])
    result[valid_k] = binomial_coeff * (p ** k_valid) * (q ** (n - k_valid))
  
  return result.item() if result.ndim == 0 else result

def cdf(n:int, p:float, x:Union[int, List[int], np.ndarray]) -> Union[float, np.ndarray]:
  "Cumulative distribution function"
  x = np.asarray(x)
  result = np.zeros_like(x, dtype=float)
  
  for i, xi in enumerate(x.flat):
    if xi < 0:
      result.flat[i] = 0
    elif xi >= n:
      result.flat[i] = 1
    else:
      k_values = np.arange(0, int(xi) + 1)
      result.flat[i] = np.sum(pmf(n,p,k_values))
  
  return result.item() if result.ndim == 0 else result

variance:Callable[[int,float], float] = lambda n,p : n * p * (1-p)

def get_stats(n:int, p:float) -> Dict[str,Any]:
  v = variance(n,p)
  stats = {
    "mean": n * p,
    "variance": v,
    "std": math.sqrt(v)
  }
  return stats

def plot_bin_dist(n:int, p:float, title:str = "") -> None:
  """Plot binomial distribution

  Args:
    n (int): number of trials
    p (float): success probability
    title (str, optional): plot title. Defaults to "".
  """
  k_values = np.arange(0, n+1)
  pmf_values = pmf(n,p, k_values)
  cdf_values = cdf(n,p, k_values)
  
  fig, (ax1, ax2) = plt.subplots(1,2, figsize=(12,5))
  
  # PMF
  ax1.bar(k_values, pmf_values, alpha=0.7, color='skyblue', edgecolor='black')
  ax1.set_xlabel('k (Number of successes)')
  ax1.set_ylabel('P(X = k)')
  ax1.set_title(f'PMF - {title}\nn={n}, p={p}')
  ax1.grid(True, alpha=0.3)
  
  # CDF
  ax2.step(k_values, cdf_values, where='post', linewidth=2, color='red')
  ax2.set_xlabel('k (Number of successes)')
  ax2.set_ylabel('P(X <= k)')
  ax2.set_title(f'CDF - {title}\nn={n}, p={p}')
  ax2.grid(True, alpha=0.3)
  
  plt.tight_layout()
  plt.show()

In [ ]:

def generate_binomial_numpy(n:int, p:float, size:int=1) -> np.ndarray:
  """Generate samples from binomial distribution using NumPy

  Args:
    n (int): number of trials
    p (float): success probability
    size (int, optional): number of samples to generate. Defaults to 1.

  Returns:
    np.ndarray: generated samples
  """
  return np.random.binomial(n,p,size)

def generate_binomial_inverse_transform(n:int, p:float, size:int=1) -> np.ndarray:
  """Generate samples using generalized inverse transform method

  Args:
    n (int): number of trials
    p (float): success probability
    size (int, optional): number of samples to generate. Defaults to 1.

  Returns:
    np.ndarray: generated samples
  """
  samples = np.zeros(size, dtype=int)
  
  for i in range(size):
    # step 1: generate Uniform(0,1)
    u = np.random.uniform(0,1)
    
    # step 2: initialize variables
    k = 0 
    prob_k = (1-p) ** n   # P(X = 0) = (1-p)^n
    cdf_k = prob_k        # F(k) = P(X <= k)
    
    # step 3: find k such that F(k-1) < U <= F(k)
    while u > cdf_k and k < n:
      k+= 1
      # calculate P(X = k) using recursive formula
      prob_k = prob_k * (p * (n-k+1)) / (k * (1-p))
      cdf_k += prob_k 
    
    samples[i] = k
  
  return samples

def compare_generation_methods(n:int, p:float, sample_size:int=1000) -> None:
  """Compare sample generation methods

  Args:
    n (int): number of trials
    p (float): success probability
    sample_size (int, optional): Defaults to 1000.
  """
  import time 
  
  # NumPy method
  start_time = time.time()
  samples_numpy = generate_binomial_numpy(n,p,sample_size)
  numpy_time = time.time() - start_time
  
  # Inverse Transform method
  start_time = time.time()
  samples_inverse = generate_binomial_inverse_transform(n,p,sample_size)
  inverse_time = time.time() - start_time
  
  stats = get_stats(n,p)
  
  # theoretical values
  theoretical_mean = stats["mean"]
  theoretical_std = stats["std"]

  # sample statistics
  print(f"--- Statistics ---")
  print(f"Theoretical\n- Mean: {theoretical_mean:.3f}\n- Std Dev: {theoretical_std:.3f}")
  
  print(f"\nNumPy\n- Mean: {np.mean(samples_numpy):.3f}\n- Std Dev: {np.std(samples_numpy):.3f}")
  print(f"Time: {numpy_time:.6f} seconds")
  
  print(f"\nInverse Transform\n- Mean: {np.mean(samples_inverse):.3f}\n- Std Dev: {np.std(samples_inverse):.3f}")
  print(f"Time: {inverse_time:.6f} seconds")
  
  # comparison plot
  fig, ax1 = plt.subplots(1, 1, figsize=(10, 5))
  
  ax1.hist(samples_numpy, 
      bins=np.arange(-0.5, n+1.5, 1), 
      alpha=0.7, 
      label='NumPy', 
      density=True, 
      color='skyblue'
  )
  ax1.hist(samples_inverse, 
      bins=np.arange(-0.5, n+1.5, 1), 
      alpha=0.7, 
      label='Inverse Transform', 
      density=True, 
      color='lightcoral'
  )
  
  # PMF teórica
  k_values = np.arange(0, n + 1)
  theoretical_pmf = pmf(n,p, k_values)
  ax1.plot(k_values, theoretical_pmf, 'ko-', label='Theoretical', markersize=4)
  
  ax1.set_xlabel('k')
  ax1.set_ylabel('Density')
  ax1.set_title('Comparison of Generation Methods')
  ax1.legend()
  ax1.grid(True, alpha=0.3)
  
  plt.tight_layout()
  plt.show()

In [ ]:
np.random.seed(42)

n,p = 10,0.3 
compare_generation_methods(n,p,1000)